# Импортирање на потребните библиотеки

In [ ]:
!pip install stellargraph

     |████████████████████████████████| 435 kB 5.2 MB/s 
     |████████████████████████████████| 462 kB 44.7 MB/s 


In [ ]:
from stellargraph.datasets import MovieLens
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, LinkEmbedding
from tensorflow.keras.layers import Reshape, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split

# Вчитување податоци

In [ ]:
# го вчитуваме податочното множество MovieLens кое е составено од корисници,
# филмови и рејтинзи на филмовите од корисниците
# корисниците и филмовите претставуваат јазли во графот, додека 
# рејтинзите претставуваат врски
dataset = MovieLens()
graph, ratings = dataset.load()

# за потребите на систем за препорака базиран на овој граф, ги трансформираме
# рејтинзите така што ако вредноста за рејтинг е поголема од 3 тогаш
# претпоставуваме дека корисникот би го препорачал тој филм (поставуваме 
# вредност 1), а во спротивно не би го препорачал (поставуваме вредност 0)
ratings['rating'] = [1 if rating > 3 else 0 for rating in ratings['rating'].values]

UnicodeDecodeError: ignored

In [ ]:
MovieLens().load()

UnicodeDecodeError: ignored

In [ ]:
# приказ на информации за графот
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 2625, Edges: 100000

 Node types:
  movie: [1682]
    Features: float32 vector, length 19
    Edge types: movie-rating->user
  user: [943]
    Features: float32 vector, length 24
    Edge types: user-rating->movie

 Edge types:
    movie-rating->user: [100000]
        Weights: all 1 (default)
        Features: none


In [ ]:
# го делиме множеството на подмножества за тренирање, валидација и тестирање
train_edges, test_edges = train_test_split(ratings, test_size=0.2)
val_edges, test_edges = train_test_split(test_edges, test_size=0.5)

In [ ]:
# за полесно користење на податоците, ги сместуваме класите во посебни низи
train_labels = train_edges['rating']
val_labels = val_edges['rating']
test_labels = test_edges['rating']

# за потребите на моделот, ги трансформираме врските така што првиот јазол
# секогаш ќе биде корисник, а вториот јазол секогаш ќе биде филм
train_edges = list(train_edges[['user_id', 'movie_id']].itertuples(index=False))
val_edges = list(val_edges[['user_id', 'movie_id']].itertuples(index=False))
test_edges = list(test_edges[['user_id', 'movie_id']].itertuples(index=False))

# Креирање модел

In [ ]:
# со цел пренесување на податоците од графот во Keras модел 
# потребно е да креира генератор
# за системи на препораки базирани на графови со HinSAGE може да се користи
# HinSAGELinkGenerator со следните аргументи
# batch_size - големина на серија
# num_samples - број на соседни јазли
# head_node_types - тип на јазли кои ги формираат врските
generator = HinSAGELinkGenerator(graph, batch_size=16, num_samples=[8, 4], head_node_types=['user', 'movie'])

In [ ]:
# креирање итератори на врски кои ќе се користат за тренирање,
# валидација и тестирање на моделот
train_gen = generator.flow(train_edges, train_labels)
val_gen = generator.flow(val_edges, val_labels)
test_gen = generator.flow(test_edges, test_labels)

In [ ]:
# за систем за препорака базиран на графови со моделот HinSAGE потребно е да 
# се креира објект кој ќе го претставува моделот, со следните аргументи
# layer_sizes - листа на големини на скриените слоеви
# activations - листа на активациски функции на скриените слоеви
# generator - генератор на податоци
# dropout - процент за регуларизација (dropout) на секој слој
hinsage = HinSAGE(layer_sizes=[16, 16], activations=['relu', 'relu'], generator=generator, dropout=0.5)

# извлекување на влезните и излезните слоеви од моделот HinSAGE
x_inp, x_out = hinsage.in_out_tensors()

# по излезниот слој од моделот HinSAGE се додаваат слоеви
# според потребите на проблемот кој се разгледува
# слојот LinkEmbedding претставува слој кој според вгнездувањата на јазлите
# кои ја формираат врската креира вгнездување на врската
# методот ip означува креирање вгнездување на врската како производ од
# вгнездувањата на јазлите
predictions = LinkEmbedding(activation='relu', method='ip')(x_out)
predictions = Reshape((-1,))(predictions)

# излезен слој за препорака
predictions = Dense(1, activation='sigmoid')(predictions)

# креирање објект кој ќе го претставува моделот со соодветните 
# влезни и излезни слоеви
model = Model(inputs=x_inp, outputs=predictions)

# пред да се тренира, потребно е моделот да се компајлира
# со поставување на аргументот loss дефинираме категориска крос-ентропија
# како функција на загуба
# со поставување на аргументот optimizer дефинираме Adam оптимизатор со рата
# на учење еднаква на 0.01
# со поставување на аргументот metrics дефинираме точност како метрика за 
# следење на перформансите на моделот при тренирање
model.compile(optimizer=Adam(learning_rate=0.01), loss=binary_crossentropy, metrics=['accuracy'])

# Тренирање

In [ ]:
# при тренирање на моделот покрај влезните и излезните податоци, потребно е 
# да се постават вредности за аргументите epochs што претставува број на 
# епохи, validation_data што претставува податоци за валидација, shuffle што
# претставува информација за тоа дали да се прави промена на редоследот на 
# податоците во различни епохи и опционално verbose со што се дефинира
# интензитет на информации за процесот на тренирање
model.fit(train_gen, epochs=3, validation_data=val_gen, shuffle=False, verbose=2)

Epoch 1/3
5000/5000 - 58s - loss: 0.6809 - accuracy: 0.5680 - val_loss: 0.6749 - val_accuracy: 0.5829
Epoch 2/3
5000/5000 - 54s - loss: 0.6772 - accuracy: 0.5769 - val_loss: 0.6701 - val_accuracy: 0.5954
Epoch 3/3
5000/5000 - 53s - loss: 0.6762 - accuracy: 0.5785 - val_loss: 0.6694 - val_accuracy: 0.5923


# Евалуација

In [ ]:
# евалуација на моделот со што се добиваат вредности за
# функцијата на загуба и точноста
model.evaluate(test_gen)

625/625 [==============================] - 5s 9ms/step - loss: 0.6653 - accuracy: 0.6066


[0.6652711629867554, 0.6065999865531921]